In [6]:
import pandas as pd
from pathlib import Path
import json
from td001_processing import postprocessing_td001
from utils import round_float_cols,unique_ordered
from config import paths
def run_enveloppe_processing(td001, td006, td007, td008):
    from td007_processing import merge_td007_tr_tv, postprocessing_td007
    from td008_processing import merge_td008_tr_tv, postprocessing_td008
    from td001_merge import merge_td001_dpe_id_envelope
    from td007_processing import agg_td007_to_td001_essential, agg_surface_envelope
    from td008_processing import agg_td008_to_td001_essential

    td008_raw_cols = td008.columns.tolist()
    td007_raw_cols = td007.columns.tolist()

    td001, td006, td007, td008 = merge_td001_dpe_id_envelope(td001=td001, td006=td006, td007=td007, td008=td008)

    td008 = merge_td008_tr_tv(td008)
    td008 = postprocessing_td008(td008)

    td007 = merge_td007_tr_tv(td007)
    td007 = postprocessing_td007(td007, td008)

    agg_td007 = agg_td007_to_td001_essential(td007)
    agg_td008 = agg_td008_to_td001_essential(td008)
    agg_surfaces = agg_surface_envelope(td007, td008)

    td001_enveloppe_agg = pd.concat([agg_td007, agg_td008, agg_surfaces], axis=1)

    td001_enveloppe_agg.index.name = 'td001_dpe_id'
    cols = [el for el in td008.columns if el not in td008_raw_cols + ['fen_lib_from_tv009',
                                                                      'fen_lib_from_tv021']]
    cols.append('td008_baie_id')
    cols = unique_ordered(cols)
    td008_p = td008[cols]
    cols = [el for el in td007.columns if
            el not in td007_raw_cols + ["qualif_surf", 'surface_paroi_opaque_calc', 'surface_paroi_totale_calc_v1',
                                        'surface_paroi_totale_calc_v2']]
    cols.append('td007_paroi_opaque_id')
    cols = unique_ordered(cols)
    td007_p = td007[cols]
    return td001_enveloppe_agg, td008_p, td007_p


def run_system_processing(td001, td006, td011, td012, td013, td014):
    from td011_td012_processing import merge_td012_tr_tv, postprocessing_td012, merge_td011_tr_tv, \
        agg_systeme_chauffage_essential
    from td013_td014_processing import merge_td013_tr_tv, postprocessing_td014, merge_td014_tr_tv, \
        agg_systeme_ecs_essential
    from td001_merge import merge_td001_dpe_id_system

    td011_raw_cols = td011.columns.tolist()
    td012_raw_cols = td012.columns.tolist()
    td013_raw_cols = td013.columns.tolist()
    td014_raw_cols = td014.columns.tolist()
    td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)
    td011 = merge_td011_tr_tv(td011)
    td012 = merge_td012_tr_tv(td012)
    td013 = merge_td013_tr_tv(td013)
    td014 = merge_td014_tr_tv(td014)

    td012 = postprocessing_td012(td012)

    cols = [el for el in td011.columns if el not in td011_raw_cols]
    cols.append('td011_installation_chauffage_id')
    cols = unique_ordered(cols)
    td011_p = td011[cols]

    cols = [el for el in td012.columns if
            el not in td012_raw_cols + ['besoin_chauffage_infer', 'gen_ch_concat_txt_desc']]
    cols.append('td012_generateur_chauffage_id')
    cols = unique_ordered(cols)
    td012_p = td012[cols]

    td001_sys_ch_agg = agg_systeme_chauffage_essential(td001, td011, td012)

    td014 = postprocessing_td014(td013, td014)

    cols = [el for el in td013.columns if el not in td013_raw_cols]
    cols.append('td013_installation_ecs_id')
    cols = unique_ordered(cols)
    td013_p = td013[cols]

    cols = [el for el in td014.columns if
            el not in td014_raw_cols + ['score_gen_ecs_lib_infer', 'gen_ecs_concat_txt_desc']]
    cols.append('td014_generateur_ecs_id')
    cols = unique_ordered(cols)
    td014_p = td014[cols]

    td001_sys_ecs_agg = agg_systeme_ecs_essential(td001, td013, td014)

    return td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg


def build_doc(annexe_dir):
    from doc_annexe import td001_annexe_enveloppe_agg_desc, td001_sys_ch_agg_desc, td001_sys_ecs_agg_desc, \
        td007_annexe_desc, td008_annexe_desc, td012_annexe_desc, td014_annexe_desc, enums_cstb,td001_annexe_generale_desc

    doc_annexe = dict()
    doc_annexe['td001_annexe_generale'] = td001_annexe_generale_desc
    doc_annexe['td001_annexe_enveloppe_agg'] = td001_annexe_enveloppe_agg_desc
    doc_annexe['td001_sys_ch_agg'] = td001_sys_ch_agg_desc
    doc_annexe['td001_sys_ecs_agg'] = td001_sys_ecs_agg_desc
    doc_annexe['td007_annexe'] = td007_annexe_desc
    doc_annexe['td008_annexe'] = td008_annexe_desc
    doc_annexe['td012_annexe'] = td012_annexe_desc
    doc_annexe['td014_annexe'] = td014_annexe_desc

    with open(annexe_dir / 'doc_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(doc_annexe, f, indent=4)

    with open(annexe_dir / 'enum_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(enums_cstb, f, indent=4)


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    build_doc(annexe_dir)
    for dept_dir in Path(data_dir).iterdir():
        print(dept_dir)
        annexe_dept_dir = annexe_dir / dept_dir.name
        annexe_dept_dir.mkdir(exist_ok=True, parents=True)
        # LOAD TABLES
        td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
        td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
        td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
        td008 = pd.read_csv(dept_dir / 'td008_baie.csv', dtype=str)
        td008 = td008.drop('td008_baie_id', axis=1)

        # ENVELOPPE PROCESSING
        td001_enveloppe_agg, td008_p, td007_p = run_enveloppe_processing(td001, td006, td007, td008)

        round_float_cols(td001_enveloppe_agg).to_csv(annexe_dept_dir / 'td001_annexe_enveloppe_agg.csv')
        round_float_cols(td007_p).to_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv')
        round_float_cols(td008_p).to_csv(annexe_dept_dir / 'td008_baie_annexe.csv')

        # SYSTEM PROCESSING

        td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
        td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
        td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
        td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

        td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                        td011, td012,
                                                                                                        td013, td014)
        round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_annexe_sys_ch_agg.csv')
        round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_annexe_sys_ecs_agg.csv')
        round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_annexe_installation_chauffage.csv')
        round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_annexe_generateur_chauffage.csv')
        round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_annexe_installation_ecs.csv')
        round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_annexe_generateur_ecs.csv')

        # add td001 processing
        postprocessing_td001(td001)[['nom_methode_dpe_norm', 'id']].rename(columns={'id': 'td001_dpe_id'}).to_csv(
            annexe_dept_dir / 'td001_annexe_generale.csv')


In [3]:
td010 = pd.read_csv(dept_dir / 'td010_pont_thermique.csv', dtype=str)


In [13]:
td007 = td007.rename(columns ={'id':'td007_paroi_opaque_id'})

In [14]:
td007_full = td007.merge(td007_p,on='td007_paroi_opaque_id',how='left')

In [33]:
td007_murs = td007_full.loc[td007_full.tr014_type_parois_opaque_id.isin(['2', '1'])].copy()


# traitements murs

In [20]:
from utils import agg_pond_top_freq,agg_pond_avg

In [34]:
epaisseur_isolation_agg = agg_pond_top_freq(td007_murs, 'epaisseur_isolation', 'surface_paroi_opaque_deperditive_infer',
                           'td001_dpe_id').to_frame('epaisseur_isolation_deper_top')

## label uniforme tv003

In [185]:
td007_murs['tv003_periode_isolation_uniforme']= td007_murs.tv003_annee_construction.astype('string')

td007_murs['tv003_label_isolation_uniforme']= td007_murs.tv003_annee_construction.astype('string')

null = td007_murs['tv003_label_isolation_uniforme'].isnull()

td007_murs.loc[null,'tv003_label_isolation_uniforme']=td007_murs.loc[null,'tv003_annee_isolation'].astype('string')

inconnu = td007_murs.tv003_mur_isole.isnull()& (~td007_murs.tv003_annee_construction.isnull())
non_isole = td007_murs.tv003_mur_isole=='0'
isole = td007_murs.tv003_mur_isole=='1'
is_construction = ~td007_murs.tv003_annee_construction.isnull()

td007_murs.loc[inconnu,'tv003_label_isolation_uniforme'] ='isol. inconnue periode constr : ' +td007_murs.loc[inconnu,'tv003_label_isolation_uniforme']
td007_murs.loc[non_isole,'tv003_label_isolation_uniforme'] ='non isolé'
td007_murs.loc[isole&is_construction,'tv003_label_isolation_uniforme'] ='isolé periode constr : ' +td007_murs.loc[isole&is_construction,'tv003_label_isolation_uniforme']
td007_murs.loc[isole&(~is_construction),'tv003_label_isolation_uniforme'] ='isolé periode isolation :'  +td007_murs.loc[isole&(~is_construction),'tv003_label_isolation_uniforme']


In [186]:
td007_murs['annee_isole_uniforme_min']=td007_murs.tv003_annee_construction_min.astype('string')
td007_murs['annee_isole_uniforme_max']=td007_murs.tv003_annee_construction_max.astype('string')
td007_murs.loc[isole,'annee_isole_uniforme_min']=td007_murs.tv003_annee_isolation_min.astype('string')
td007_murs.loc[isole,'annee_isole_uniforme_max']=td007_murs.tv003_annee_isolation_max.astype('string')


In [187]:
float_cols = ['coefficient_transmission_thermique_paroi_non_isolee','coefficient_transmission_thermique_paroi','epaisseur_isolation','resistance_thermique_isolation']

In [188]:
td007_murs.tv003_label_isolation_uniforme.value_counts()

isol. inconnue periode constr : 1948 à 1974         33055
isolé periode isolation :à partir de 2006            9820
isolé periode isolation :de 1989 à 2000              6790
non isolé                                            5153
isolé periode constr : à partir de 2000              5063
isolé periode isolation :de 2001 à 2005              4001
isolé periode constr : jusqu'à 1988                  3675
isolé periode isolation :de 1983 à 1988              3217
isolé periode isolation :de 1978 à 1982              2448
isolé periode isolation :de 1975 à 1977              2015
isol. inconnue periode constr : 1988 à 2000           682
isol. inconnue periode constr : 1977 à 1982           499
isol. inconnue periode constr : à partir de 2006      475
isol. inconnue periode constr : 1974 à 1977           462
isolé periode constr : de 1989 à 2000                 405
isol. inconnue periode constr : 1982 à 1988           380
isol. inconnue periode constr : 2000 à 2005           231
Name: tv003_la

## label méthode calcul  U

In [171]:
td007_murs[float_cols] = td007_murs[float_cols].astype(float)

In [172]:
td007_murs['meth_calc_U']='INCONNUE'

In [173]:
U = td007_murs.coefficient_transmission_thermique_paroi.round(2)
U_non_isolee = td007_murs.coefficient_transmission_thermique_paroi_non_isolee.round(2)
bool_U_egal_0=U.round(2)==0.00
bool_U_U0 =U.round(2)==U_non_isolee.round(2)
bool_U_2 = U.round(2)>=2|non_isole
bool_U_U0 = bool_U_U0 & (~bool_U_2)
bool_U_U0_auto_isol = bool_U_U0&(U_non_isolee<1)
bool_U_brut = (U<=1)&(~bool_U_U0)
bool_U_brut_non_isole = (U>1)&(~bool_U_U0)

bool_U_par_e=td007_murs.epaisseur_isolation>0
bool_U_par_r=td007_murs.resistance_thermique_isolation>0

bool_U_tv003 = ~td007_murs.tv003_label_isolation_uniforme.isnull()

In [174]:
td007_murs.loc[bool_U_brut,'meth_calc_U']='U SAISI DIRECTEMENT : ISOLE'
td007_murs.loc[bool_U_brut_non_isole,'meth_calc_U']='U SAISI DIRECTEMENT : NON ISOLE'
td007_murs.loc[bool_U_par_e,'meth_calc_U']='EPAISSEUR ISOLATION SAISIE'
td007_murs.loc[bool_U_par_r,'meth_calc_U']='RESISTANCE ISOLATION SAISIE'
td007_murs.loc[bool_U_2,'meth_calc_U']='MUR NON ISOLE U=2'
td007_murs.loc[bool_U_U0,'meth_calc_U']='MUR NON ISOLE U<2'
td007_murs.loc[bool_U_U0_auto_isol,'meth_calc_U']='STRUCTURE ISOLANTE (ITR) U<1'
td007_murs.loc[inconnu,'meth_calc_U']='PAR DEFAUT PERIODE : ISOLATION INCONNUE'
td007_murs.loc[isole,'meth_calc_U']='PAR DEFAUT PERIODE : ISOLE'
td007_murs.loc[isole,'meth_calc_U']='PAR DEFAUT PERIODE : ISOLE'
td007_murs.loc[bool_U_egal_0,'meth_calc_U']='ERREUR : U=0'




## label isolatoin

In [206]:
td007_murs['isolation']='NON ISOLE'
is_isole = ~td007_murs.meth_calc_U.str.contains('NON ISOLE|INCONNUE')
td007_murs.loc[is_isole,'isolation']='ISOLE SAISI'
is_isole_defaut = is_isole &(td007_murs.meth_calc_U.str.contains('DEFAUT'))
td007_murs.loc[is_isole_defaut,'isolation']='ISOLE DEFAUT PRE 1988'

inconnu = td007_murs.meth_calc_U.str.contains('INCONNUE')
post_88=td007_murs.loc[isole,'annee_isole_uniforme_min']>="1988"

td007_murs.loc[inconnu,'isolation']='ISOLATION INCONNUE (DEFAUT)'

td007_murs.loc[(inconnu|is_isole_defaut)&post_88,'isolation']='ISOLE DEFAUT POST 1988'

is_isole_struc = is_isole &(td007_murs.meth_calc_U.str.contains('STRUCTURE'))
td007_murs.loc[is_isole_struc,'isolation']='STRUCTURE ISOLANTE (ITR)'

is_err = td007_murs.meth_calc_U.str.contains('ERREUR')

td007_murs.loc[is_err,'isolation']='NONDEF'

## label adjacence

In [244]:
td007_murs['type_adjacence']='EXTERIEUR'

not_null = ~td007_murs.tv002_local_non_chauffe.isnull()

is_lnc = td007_murs.tv001_code.astype('string')>'TV001_004'


td007_murs.loc[is_lnc,'type_adjacence']='LNC'

is_adj=td007_murs.tv001_code=='TV001_004'

td007_murs.loc[is_adj,'type_adjacence']='BAT_ADJ'

In [230]:
td007_murs.tv001_valeur=td007_murs.tv001_valeur.astype(float)

In [232]:
td007_murs.groupby('type_adjacence').tv001_valeur.describe()

,count,mean,std,min,25%,50%,75%,max
type_adjacence,,,,,,,,
LC,5478.0,0.000000,0.000000,0.00,0.00,0.0,0.00,0.0
LNC,24653.0,0.718324,0.268441,0.05,0.55,0.8,0.95,1.0
exterieur,85746.0,0.955310,0.180115,0.05,1.00,1.0,1.00,1.0


In [233]:
td007_murs.groupby('type_adjacence').b_infer.describe()

,count,mean,std,min,25%,50%,75%,max
type_adjacence,,,,,,,,
LC,3880.0,0.000000,0.000000,0.000,0.000,0.000,0.00,0.0
LNC,36216.0,0.714148,0.306458,-0.684,0.550,0.834,0.95,5.0
exterieur,88509.0,0.931962,0.210463,-0.928,0.986,1.000,1.00,5.0


In [217]:
td007_murs.tv001_valeur.min()

'-0.02'

In [224]:
td007_murs.loc[is_0].deperdition_thermique.astype(float).mean()

0.0

In [218]:
(td007_murs.b_infer<0.1).mean()

0.06690130645114568

In [142]:
td007_murs.meth_calc_U.value_counts()

PAR DEFAUT PERIODE : ISOLE                 37412
EPAISSEUR ISOLATION SAISIE                 36646
PAR DEFAUT PERIODE : ISOLATION INCONNUE    35784
MUR NON ISOLE U=2                          18779
RESISTANCE ISOLATION SAISIE                 4752
U SAISI DIRECTEMENT : ISOLE                 2855
ERREUR : U=0                                1565
STRUCTURE ISOLANTE (ITR) U<1                 291
Name: meth_calc_U, dtype: int64

In [94]:
not_tv004=td007_murs.tv004_code.isnull()

In [97]:
td007_murs.coefficient_transmission_thermique_paroi_non_isolee

0         0.00
1         0.00
2         0.00
3         0.00
6         0.00
          ... 
237478     NaN
237479     NaN
237480     NaN
237481     NaN
237482     NaN
Name: coefficient_transmission_thermique_paroi_non_isolee, Length: 138084, dtype: object

In [99]:
td007_murs.loc[not_tv004][['epaisseur_isolation','resistance_thermique_isolation','coefficient_transmission_thermique_paroi_non_isolee','coefficient_transmission_thermique_paroi']]

,epaisseur_isolation,resistance_thermique_isolation,coefficient_transmission_thermique_paroi_non_isolee,coefficient_transmission_thermique_paroi
232,0.0,0.00,0.00,0.40
570,0.0,0.00,0.00,0.40
748,0.0,0.27,0.00,1.42
749,0.0,1.93,25.20,0.43
750,0.0,2.96,0.00,0.29
...,...,...,...,...
237333,0.0,0.00,10.00,2.50
237400,0.0,0.00,0.00,0.98
237401,0.0,0.00,13.29,2.00
237438,0.0,0.00,NaN,0.00


In [90]:
bool_tv004=~td007_murs.tv004_code.isnull()

In [89]:
td007_murs.loc[bool_U_par_e]

,td007_paroi_opaque_id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,...,surfacexnb_baie_calc_sum,nb_baie_calc,surface_paroi_opaque_infer,surface_paroi_opaque_deperditive_infer,b_infer,surface_paroi_opaque_exterieur_infer,3a5326e0-8a53-4c5b-9943-b655ec8ade26,tv003_annee_isolation_uniforme,tv003_label_isolation_uniforme,meth_calc_U
0,688,135,1,Mur 1,12.33,1,NaN,0.35,0.00,NaN,...,1.00,1.0,35.22,35.22,1.000,35.22,35.22,nan,<NA>,INCONNUE
1,689,135,1,Mur 2,11.61,1,NaN,0.35,0.00,NaN,...,7.20,3.0,33.17,33.17,1.000,33.17,33.17,nan,<NA>,INCONNUE
2,690,135,1,Mur 3,13.76,1,NaN,0.35,0.00,NaN,...,8.07,2.0,39.31,39.31,1.000,39.31,39.31,nan,<NA>,INCONNUE
3,691,135,1,Mur 4,10.87,1,NaN,0.35,0.00,NaN,...,7.21,3.0,31.06,31.06,1.000,31.06,31.06,nan,<NA>,INCONNUE
6,1490,307,1,Mur extérieur Sud,10.62,NaN,NaN,0.43,0.00,NaN,...,5.81,2.0,24.89,24.89,0.992,24.89,24.89,nan,<NA>,INCONNUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237473,24113832,5419402,1,Mur 4,4.75,1,NaN,0.32,0.00,1,...,3.12,2.0,14.75,14.75,1.006,14.75,14.75,1948 à 1974,isol. inconnue periode constr : 1948 à 1974,INCONNUE
237474,24113833,5419402,1,Mur 5,5.33,1,NaN,0.32,0.00,1,...,2.58,1.0,16.57,16.57,1.005,16.57,16.57,1948 à 1974,isol. inconnue periode constr : 1948 à 1974,INCONNUE
237475,24113834,5419402,1,Mur 6,3.55,1,NaN,0.32,0.00,1,...,2.58,1.0,11.04,11.04,1.005,11.04,11.04,1948 à 1974,isol. inconnue periode constr : 1948 à 1974,INCONNUE
237476,24113835,5419402,1,Mur 7,1.74,1,NaN,0.32,0.00,1,...,1.75,1.0,5.40,5.40,1.007,5.40,5.40,1948 à 1974,isol. inconnue periode constr : 1948 à 1974,INCONNUE


In [72]:
bool_U_sup_2=td007_murs.coefficient_transmission_thermique_paroi.round(3)>=2 ### Pour U = 2
bool_U_inf_2=td007_murs.coefficient_transmission_thermique_paroi.round(3)<2
bol_U_sup_0=td007_murs.coefficient_transmission_thermique_paroi.round(2)>0.00

# Erreur à sortir
bool_U_egal_0=td007_murs.coefficient_transmission_thermique_paroi.round(2)==0.00

####
# cas e donnée
bool_U_inf_U0=td007_murs.coefficient_transmission_thermique_paroi.round(2)<td007_murs.coefficient_transmission_thermique_paroi_non_isolee.round(2)
bool_U_par_e=td007_murs.epaisseur_isolation>0

# cas R donnée
bool_U_inf_U0=td007_murs.coefficient_transmission_thermique_paroi.round(2)<td007_murs.coefficient_transmission_thermique_paroi_non_isolee.round(2)
bool_U_par_R=td007_murs.resistance_thermique_isolation>0

# Cas U = U0
bool_U_egal_U0=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.coefficient_transmission_thermique_paroi_non_isolee.round(2)
bool_U_dif_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv003_Umur
bool_U_dif_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv004_Umur0
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0

# Cas U = Utv statut isolation : Pas d'isolation
bool_U_egal_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.tv003_Umur.round(2)
bool_U_dif_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_Utv_existe=td007_murs.tv003_Umur>0
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_isol_NON = td007_murs.tv003_umur_id == 'TV003_043'

# Cas U = Utv statut isolation : inconnue
bool_U_egal_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.tv003_Umur.round(2)
bool_U_dif_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_Utv_existe=td007_murs.tv003_Umur>0
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_isol_INC = td007_murs.tv003_umur_id < 'TV003_043'

# Cas U = Utv statut isolation : année d'isolation fournie (rénovation?)
bool_U_egal_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.tv003_Umur.round(2)
bool_U_dif_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_Utv_existe=td007_murs.tv003_Umur>0
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!=td007_murs.tv004_Umur0.round(2)
bool_isol_OUI = td007_murs.tv003_umur_id > 'TV003_043'

# Cas U = Utv0 : element structure performant
bool_U_egal_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.tv004_Umur0.round(2)
bool_U_dif_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv003_Umur.round(2)
bool_Utv0_existe=td007_murs.tv004_Umur0>0
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_Utv0_PERF=td007_murs.tv004_Umur0<=1

# Cas U = Utv0 : element structure non performant
bool_U_egal_Utv0=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.tv004_Umur0.round(2)
bool_U_dif_Utv=td007_murs.coefficient_transmission_thermique_paroi.round(2)!=td007_murs.tv003_Umur.round(2)
bool_Utv0_existe=td007_murs.tv004_Umur0>0
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_Utv0_NON_PERF=td007_murs.tv004_Umur0>1

# Cas U donné
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_Utv_non_def=td007_murs.tv003_Umur.isnull()
bool_Utv0_non_def=td007_murs.tv004_Umur0.isnull()
bool_U0_non_def=td007_murs.coefficient_transmission_thermique_paroi_non_isolee<=0


# Cas U=U0 performant U<1
bool_U_egal_U0=td007_murs.coefficient_transmission_thermique_paroi.round(2)==td007_murs.coefficient_transmission_thermique_paroi_non_isolee.round(2)
bool_Utv_non_def=td007_murs.tv003_Umur.isnull()
bool_Utv0_non_def=td007_murs.tv004_Umur0.isnull()
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_U_inf_1=td007_murs.coefficient_transmission_thermique_paroi<1 


### Cas U>=2
bool_U_sup_2=td007_murs.coefficient_transmission_thermique_paroi>=2  ### Pour cas U >= 2 et R donnée
# cas e donnée
bool_U_par_e=td007_murs.epaisseur_isolation>0

# cas R donnée
bool_U_par_R=td007_murs.resistance_thermique_isolation>0

# Cas U = U0 (à revoir)
bool_U0_sup_2=td007_murs.coefficient_transmission_thermique_paroi_non_isolee>=2
bool_Utv0_non_def=td007_murs.tv004_Umur0.isnull()
bool_Utv_non_def=td007_murs.tv003_Umur.isnull()
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0

# Cas U = Utv statut isolation : non
bool_Utv_sup_2=td007_murs.tv003_Umur>=2
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!= td007_murs.tv004_Umur0.round(2)
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_isol_NON = td007_murs.tv003_umur_id == 'TV003_043'

# Cas U = Utv statut isolation : inconnue
bool_Utv_sup_2=td007_murs.tv003_Umur>=2
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!= td007_murs.tv004_Umur0.round(2)
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_isol_INC = td007_murs.tv003_umur_id < 'TV003_043'

# Cas U = Utv statut isolation : oui
bool_Utv_sup_2=td007_murs.tv003_Umur>=2
bool_Utv_dif_Utv0=td007_murs.tv003_Umur.round(2)!= td007_murs.tv004_Umur0.round(2)
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_isol_OUI = td007_murs.tv003_umur_id > 'TV003_043'

# Cas U = Utv = Utv0
bool_Utv_sup_2=td007_murs.tv003_Umur>=2
bool_Utv_egal_Utv0=td007_murs.tv003_Umur.round(2)== td007_murs.tv004_Umur0.round(2)
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0

# Cas U = Utv0
bool_Utv0_sup_2=td007_murs.tv004_Umur0>=2
bool_Utv_non_def=td007_murs.tv003_Umur.isnull()
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0

# Cas U donné
bool_e_non_def=td007_murs.epaisseur_isolation<=0
bool_R_non_def=td007_murs.resistance_thermique_isolation<=0
bool_Utv_non_def=td007_murs.tv003_Umur.isnull()
bool_Utv0_non_def=td007_murs.tv004_Umur0.isnull()
bool_U0_non_def=td007_murs.coefficient_transmission_thermique_paroi_non_isolee<=0


TypeError: can't multiply sequence by non-int of type 'float'

In [84]:
td007_murs.coefficient_transmission_thermique_paroi.describe()

count    138084.000000
mean          0.736344
std           0.650216
min           0.000000
25%           0.350000
50%           0.400000
75%           0.860000
max           3.000000
Name: coefficient_transmission_thermique_paroi, dtype: float64

In [70]:
td007_murs.loc[not_default][['epaisseur_isolation','resistance_thermique_isolation','coefficient_transmission_thermique_paroi']]

,epaisseur_isolation,resistance_thermique_isolation,coefficient_transmission_thermique_paroi
0,10.00,0.00,0.35
1,10.00,0.00,0.35
2,10.00,0.00,0.35
3,10.00,0.00,0.35
6,5.00,0.00,0.43
...,...,...,...
237360,10.00,0.00,0.54
237361,10.00,0.00,0.54
237401,0.00,0.00,2.00
237438,0.00,0.00,0.00


In [42]:
td007_murs.tv003_mur_isole.value_counts().index[0]

'1'

In [37]:
td007_murs.tv003_annee_isolation_uniforme.value_counts()

nan                 93157
1948 à 1974         33055
à partir de 2000     5063
jusqu'à 1988         3675
1988 à 2000           682
1977 à 1982           499
à partir de 2006      475
1974 à 1977           462
de 1989 à 2000        405
1982 à 1988           380
2000 à 2005           231
Name: tv003_annee_isolation_uniforme, dtype: int64

In [22]:
epaisseur_isolation_agg

,epaisseur_isolation_deper_top
td001_dpe_id,
999746,0.00
999726,0.00
999660,10.00
99948,0.00
99943,0.00
...,...
100207,0.00
1002065,0.00
1001934,0.00


In [17]:
td007_murs.epaisseur_isolation


0         10.00
1         10.00
2         10.00
3         10.00
6          5.00
          ...  
237478     0.00
237479     0.00
237480     0.00
237481     0.00
237482     0.00
Name: epaisseur_isolation, Length: 138084, dtype: object